### Import document avec PDFPumbler

In [ ]:
import pdfplumber

path = "./docs/pse2024_public.pdf"

def extract_pdf(path):
    texte = ""
    with pdfplumber.open(path) as pdf:
        for page in pdf.pages:
            texte += page.extract_text()
    return texte


In [16]:
docs = extract_pdf(path)

### Importer document avec PyPDFLoader

In [42]:
from langchain_community.document_loaders import PyPDFLoader
import pypdf
import os

# Merge all PDF into a single PDF
writer = pypdf.PdfWriter()

for pdf in os.listdir("./docs"):
    writer.append(f'./docs/{pdf}')

writer.write("./docs/all_docs.pdf")
writer.close()

# Load PDF
loader = PyPDFLoader("./docs/all_docs.pdf")
docs = loader.load()

In [43]:
# Afficher le texte de la première page
print("Texte de la première page :")
print(docs[0].page_content[:500])  # Affiche les 500 premiers caractères


Texte de la première page :
Fédération nationale de protection
civile
Logo de la Protection Civile.
H i s t o i r e 
Fondation 14 décembre 1965
Origine Directive du 18 mars 1964 de
Georges Pompidou
C a d r e 
Sigle FNPC
Zone d'activité France, Monde entier
Forme juridique Association
Objectif Postes de secours,
formation aux premiers
secours,
aide humanitaire et sociale
Siège Asnières-sur-Seine
Pays
  France
O r g a n i s a t i o n 
Membres 32 000[Quand ?]
Président François Richez
Vice-président Jean-François Cueille,
Lin


### Decoupage en chuck

In [44]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200
)

chunks = text_splitter.split_documents(docs)

### Embedding documents and vectore store

In [46]:
from langchain_community.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings

embedding_model = HuggingFaceEmbeddings(
    model_name="sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2")

# Créer la base FAISS directement
vectorstore = FAISS.from_documents(docs, embedding_model)  
#sauvegarde de l'index

vectorstore.save_local("./data/faiss_index")

In [47]:
# Effectuer une recherche
query = "Que faire en cas d'accouchement inopiné ?"
docs = vectorstore.similarity_search(query, k=2)  # k = nombre de résultats

print(f"Résultats pour la requête : '{query}'")
for i, doc in enumerate(docs):
    print(f"Résultat {i+1} : {doc.page_content}")


Résultats pour la requête : 'Que faire en cas d'accouchement inopiné ?'
Résultat 1 : 219 
[07AC03 / 12-2023] PSE② 
Accouchement inopiné 
Définition  
L’accouchement inopiné est l’acte de mettre au monde un nouveau -né hors d’une maternité. Il peut 
survenir avant l’arrivée des secours ou en leur présence. 
Normalement, l’accouchement survient lors du 9 ème mois de grossesse, soit entre 37  et 41 semaines 
révolues d’aménorrhée (absence de règles).  Avant 37 semaines d’aménorrhée , on parle d’accouchement 
prématuré. 
On appelle parturiente une femme enceinte lorsque le travail a commencé. 
L’accouchement se déroule en trois étapes : 
 Le travail 
À la fin de la grossesse, l’utérus commence à se contracter, de plus en plus régulièrement et de plus en 
plus fort. La partie basse de l’utérus commence à s’ouvrir pour pouvoir laisser passer l’enfant. 
L’ouverture se traduit par la sortie de glaires sanguinolentes puis d’un liquide clair, c’est la rupture de la 
poche des eaux. Elle corresp

### RAG

In [2]:
from langchain.agents.middleware import dynamic_prompt, ModelRequest
from langchain.agents import create_agent
from langchain_mistralai import ChatMistralAI
from langchain_community.vectorstores import FAISS
from dotenv import load_dotenv
import os
from langchain_huggingface import HuggingFaceEmbeddings

embedding_model = HuggingFaceEmbeddings(
    model_name="sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2")

vectorstore = FAISS.load_local(
    folder_path="./data/faiss_index",
    embeddings=embedding_model,
    allow_dangerous_deserialization=True
    )

# Initialiser le LLM
load_dotenv()
API_KEY = os.getenv("API_KEY")
llm = ChatMistralAI(mistral_api_key=API_KEY, model="mistral-small")

@dynamic_prompt
def prompt_with_context(request: ModelRequest) -> str:
    """Inject context into state messages."""
    last_query = request.state["messages"][-1].text
    retrieved_docs = vectorstore.similarity_search(last_query)

    docs_content = "\n\n".join(doc.page_content for doc in retrieved_docs)

    system_message = (
        "Tu es un sauveteur professionnel et tu donnes des instructions à des sauventeurs apprentis. Utilise ce contexte pour répondre:"
        f"\n\n{docs_content}"
    )

    return system_message

agent = create_agent(llm, tools=[], middleware=[prompt_with_context])

d:\PROJET\RAG\venv\Lib\site-packages\langchain_core\_api\deprecation.py:26: UserWarning: Core Pydantic V1 functionality isn't compatible with Python 3.14 or greater.
  from pydantic.v1.fields import FieldInfo as FieldInfoV1
d:\PROJET\RAG\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:

query = "Comment réagir en cas d'accouchement inopiné ?"

for step in agent.stream(
    {"messages": [{"role": "user", "content": query}]},
    stream_mode="values",
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

Comment réagir en cas d'accouchement inopiné ?
================================== Ai Message ==================================

En tant que sauveteur professionnel, voici comment vous devez réagir en cas d'accouchement inopiné, en suivant les protocoles PSE2 :

### **1. Installation de la femme enceinte**
- **Installez la femme enceinte sur le côté** (position latérale de sécurité). **Évitez absolument de la placer sur le dos**, car cela peut comprimer la veine cave inférieure et réduire l'oxygénation du fœtus.

### **2. Réalisation d'un bilan**
- **Effectuez un bilan complet** de la parturiente en utilisant le protocole des 4 regards (observation, auscultation, palpation, interrogatoire).
- **Interrogez la parturiente, son entourage ou consultez le carnet de maternité** pour recueillir les informations suivantes :
  - Suivi ou non de la grossesse.
  - Nombre d’accouchements déjà réalisés et leur déroule

In [ ]:
query = "Comment réagir en cas d'accouchement inopiné ?"
response = ""
for step in agent.stream(
    {"messages": [{"role": "user", "content": query}]},
    stream_mode="values"
    ):

    print(step["messages"])
    reponse = step["messages"][-1].text
    print('reponse:', reponse)

[HumanMessage(content="Comment réagir en cas d'accouchement inopiné ?", additional_kwargs={}, response_metadata={}, id='6b8cd196-61df-4581-a5e1-863e3a2683f6')]
[HumanMessage(content="Comment réagir en cas d'accouchement inopiné ?", additional_kwargs={}, response_metadata={}, id='6b8cd196-61df-4581-a5e1-863e3a2683f6'), AIMessage(content="En tant que sauveteur secouriste, voici comment réagir en cas d'accouchement inopiné, en suivant les protocoles établis :\n\n### 1. **Installation de la Femme Enceinte**\n- **Installer la femme enceinte sur le côté** : L'installation sur le dos est à proscrire.\n- **Réaliser un bilan de la femme enceinte** : Effectuer un bilan complet en utilisant le protocole du 4ème regard.\n\n### 2. **Interrogatoire et Recueil d'Informations**\nLors de l'interrogatoire, recueillir les éléments spécifiques suivants :\n- Le suivi ou non de la grossesse.\n- Le nombre d'accouchements déjà réalisés et leur déroulement.\n- La date et le lieu prévus de l'accouchement.\n- S'

In [11]:
query = "Quand a été créer la protection civile ?"
response = ""
for step in agent.stream(
    {"messages": [{"role": "user", "content": query}]},
    stream_mode="values"
    ):

    print(step["messages"])
    reponse = step["messages"][-1].text
    print('reponse:', reponse)

[HumanMessage(content='Quand a été créer la protection civile ?', additional_kwargs={}, response_metadata={}, id='59974b46-0af3-481f-afee-f0779ace9db8')]
reponse: Quand a été créer la protection civile ?
[HumanMessage(content='Quand a été créer la protection civile ?', additional_kwargs={}, response_metadata={}, id='59974b46-0af3-481f-afee-f0779ace9db8'), AIMessage(content="La Fédération Nationale de Protection Civile (FNPC) a été créée en 1967. Elle est une association loi de 1901, reconnue d'utilité publique, qui œuvre dans le domaine de la protection des populations civiles en temps de paix et de guerre. La FNPC se distingue des services de l'État investis de missions opérationnelles en matière de sécurité civile, mais elle contribue activement à la formation aux premiers secours, à l'action humanitaire et sociale, ainsi qu'au renfort des secours publics.\n\nEn tant que sauveteur professionnel, il est important de rappeler à vos apprentis secouristes que la protection civile est un 

In [ ]:
agent.invoke

In [64]:
step['messages'][0] #.content

HumanMessage(content='La protection civile est-elle une entreprise privée ?', additional_kwargs={}, response_metadata={}, id='7abcf6e5-ee95-4b83-a3ed-a94bf3976493')

In [50]:
query = "La protection civile est-elle une entreprise privée ?"
for step in agent.stream(
    {"messages": [{"role": "user", "content": query}]},
    stream_mode="values",
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

La protection civile est-elle une entreprise privée ?
================================== Ai Message ==================================

Non, la protection civile n'est pas une entreprise privée. Elle est une association loi de 1901, ce qui signifie qu'elle est à but non lucratif et fonctionne sur la base du bénévolat. Les secouristes et moniteurs de la fédération nationale de protection civile sont des bénévoles, tout comme ceux des autres associations agréées de sécurité civile (AASC).

Il est important de ne pas confondre cette association avec des services de l'État investis de missions opérationnelles en matière de sécurité civile. La protection civile est une association qui œuvre dans le cadre de la sécurité civile, mais elle n'est pas un service de l'État.
